In [196]:
# ensemble_attn_mask_per_layer [40, 128, 256, 5] to [N*H * TDST//BLOCK_SIZE_Q, mask_k//BLOCK_SIZE_K * ensemble_model_n]
import random
import numpy as np
import torch
import os
import math

# from ...hip.utils import seed
def seed(seed=50):
    torch.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed) # if use multi-GPU
    os.environ["PYTHONHASHSEED"] = str(seed)

ensemble_method_final_inter_thresh = 4
ensemble_method_final_bdd_mask_k = 0
ensemble_model_n = 5
seed()

ensemble_attn_mask_per_layer = torch.randint(1, 12, (2,3,3,5))
print(ensemble_attn_mask_per_layer)
_N_H, TDST_BQ, MASK_K_BK, MODEL_N = ensemble_attn_mask_per_layer.shape

origin_sparsity = (torch.sum(ensemble_attn_mask_per_layer < 32000)//ensemble_model_n).item()

ensemble_attn_mask_per_layer = ensemble_attn_mask_per_layer.view(_N_H * TDST_BQ, MASK_K_BK * MODEL_N)



tensor([[[[ 4, 11,  7,  9,  7],
          [ 7,  9,  1,  3,  7],
          [ 5, 10, 10,  3,  2]],

         [[ 9,  5,  3,  9,  8],
          [ 2,  1, 10,  3,  5],
          [ 4,  2,  8,  8,  9]],

         [[10,  3,  5,  4, 10],
          [10,  1, 10,  2,  7],
          [ 2,  7, 11,  7,  6]]],


        [[[ 9,  6,  7,  8,  9],
          [ 2,  8,  9,  4,  6],
          [11,  5, 10,  9,  6]],

         [[ 7, 11, 11,  7,  1],
          [ 8,  9,  5,  6,  7],
          [10,  6,  3,  1,  5]],

         [[ 1,  4,  9,  8, 10],
          [ 8, 10,  4, 11,  7],
          [ 8,  3,  6,  4,  5]]]])


In [197]:
print(ensemble_attn_mask_per_layer)
print(ensemble_attn_mask_per_layer.shape)

tensor([[ 4, 11,  7,  9,  7,  7,  9,  1,  3,  7,  5, 10, 10,  3,  2],
        [ 9,  5,  3,  9,  8,  2,  1, 10,  3,  5,  4,  2,  8,  8,  9],
        [10,  3,  5,  4, 10, 10,  1, 10,  2,  7,  2,  7, 11,  7,  6],
        [ 9,  6,  7,  8,  9,  2,  8,  9,  4,  6, 11,  5, 10,  9,  6],
        [ 7, 11, 11,  7,  1,  8,  9,  5,  6,  7, 10,  6,  3,  1,  5],
        [ 1,  4,  9,  8, 10,  8, 10,  4, 11,  7,  8,  3,  6,  4,  5]])
torch.Size([6, 15])


In [198]:
if ensemble_method_final_bdd_mask_k == 1:
    pass
    # ensemble_indices_k_size = mask_k//block_size_k
else:
    # TODO set to max possible memory : change it more efficiently
    ensemble_indices_k_size = MASK_K_BK * MODEL_N 

# TODO: Is it better to start plain and concatenate?
# ensembled_indices = torch.full((_N_H*TDST_BQ, ensemble_indices_k_size), 32000) # change to (N_H, TDST_BQ, ensemble_indices_k_size)

# k_size_max = 0

# NOTE per_query_token_cnt_diclist is just for analysis
if os.environ.get('ENSEMBLE_AGREE_DICLIST', '0') == '1':
    per_query_token_cnt_diclist = []

# ensemble_attn_mask_per_layer : [N*H * TDST//BLOCK_SIZE_Q, mask_k//BLOCK_SIZE_K * ensemble_model_n]
unique_x, indices, unique_cnt = torch.unique(ensemble_attn_mask_per_layer, sorted=False, return_inverse=True, return_counts=True)
# indices -= indices.min(dim=1, keepdims=True)[0]




unique_x, unique_cnt

(tensor([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11]),
 tensor([ 6,  6,  7,  7,  8,  7, 12,  9, 11, 11,  6]))

In [199]:
# cnt_x = (unique_x[None, None, :] == ensemble_attn_mask_per_layer[:, :, None]).long().sum(1)
N, K = ensemble_attn_mask_per_layer.shape

cnt_xs = []
chunk_n = 64
for icn in range(int(math.ceil(N / chunk_n))):
    cnt_xs.append(
        (unique_x[None, None, :] == ensemble_attn_mask_per_layer[icn*chunk_n:icn*chunk_n+chunk_n, :, None]).long().sum(1)
    )
cnt_x = torch.cat(cnt_xs, dim=0)


result = torch.full_like(ensemble_attn_mask_per_layer, 32000)
result = result.scatter_(1, indices.clamp(0, K-1), ensemble_attn_mask_per_layer)


# t = result[:, :, None] == unique_x[None, None, :]
# t = t * torch.arange(len(unique_x), device=t.device)[None, None, :]
# t = t.sum(-1)
N, K = result.shape
ts = []
chunk_n = 64
for icn in range(int(math.ceil(N / chunk_n))):
    t = result[icn*chunk_n:icn*chunk_n+chunk_n, :, None] == unique_x[None, None, :]
    t = t * torch.arange(len(unique_x), device=t.device)[None, None, :]
    t = t.sum(-1)
    ts.append(t)
t = torch.cat(ts, dim=0)

# torch.Size([4096, 1280]) torch.Size([64, 1280]) torch.Size([4096, 2094])
# print(result.shape, t.shape, cnt_x.shape)
result_cnt = torch.where(result < 32000, cnt_x.gather(-1, t), -9999999)

print(unique_x)
print(unique_cnt)
print(result)
print(result_cnt)

tensor([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])
tensor([ 6,  6,  7,  7,  8,  7, 12,  9, 11, 11,  6])
tensor([[    1,     2,     3,     4,     5, 32000,     7, 32000,     9,    10,
            11, 32000, 32000, 32000, 32000],
        [    1,     2,     3,     4,     5, 32000, 32000,     8,     9,    10,
         32000, 32000, 32000, 32000, 32000],
        [    1,     2,     3,     4,     5,     6,     7, 32000, 32000,    10,
            11, 32000, 32000, 32000, 32000],
        [32000,     2, 32000,     4,     5,     6,     7,     8,     9,    10,
            11, 32000, 32000, 32000, 32000],
        [    1, 32000,     3, 32000,     5,     6,     7,     8,     9,    10,
            11, 32000, 32000, 32000, 32000],
        [    1, 32000,     3,     4,     5,     6,     7,     8,     9,    10,
            11, 32000, 32000, 32000, 32000]])
tensor([[       1,        1,        2,        1,        1, -9999999,        4,
         -9999999,        2,        2,        1, -9999999, -9999999, -

In [200]:
# ensemble_sorted : [N*H * TDST//BLOCK_SIZE_Q, mask_k//BLOCK_SIZE_K * ensemble_model_n]
# ensemble_cnt_sorted : [N*H * TDST//BLOCK_SIZE_Q, mask_k//BLOCK_SIZE_K * ensemble_model_n]
ensemble_cnt_sorted, indices = torch.sort(result_cnt, dim=-1, descending=True)
ensemble_sorted = result.gather(-1, indices)
mask = ensemble_cnt_sorted >= ensemble_method_final_inter_thresh

ensemble_filtered = torch.where(mask, ensemble_sorted, torch.tensor(32000, device=mask.device))
ensemble_cnt_filtered = torch.where(mask, ensemble_cnt_sorted, torch.tensor(32000, device=mask.device))

print(ensemble_method_final_inter_thresh)
print(ensemble_sorted)
print(ensemble_cnt_sorted)
# print(mask)
print(ensemble_filtered)
print(ensemble_cnt_filtered)

4
tensor([[    7,     3,     9,    10,     1,     2,     4,     5,    11, 32000,
         32000, 32000, 32000, 32000, 32000],
        [    8,     9,     2,     3,     5,     1,     4,    10, 32000, 32000,
         32000, 32000, 32000, 32000, 32000],
        [   10,     7,     2,     1,     3,     4,     5,     6,    11, 32000,
         32000, 32000, 32000, 32000, 32000],
        [    9,     6,     8,     2,     4,     5,     7,    10,    11, 32000,
         32000, 32000, 32000, 32000, 32000],
        [    7,     1,     5,     6,    11,     3,     8,     9,    10, 32000,
         32000, 32000, 32000, 32000, 32000],
        [    4,     8,    10,     1,     3,     5,     6,     7,     9,    11,
         32000, 32000, 32000, 32000, 32000]])
tensor([[       4,        2,        2,        2,        1,        1,        1,
                1,        1, -9999999, -9999999, -9999999, -9999999, -9999999,
         -9999999],
        [       3,        3,        2,        2,        2,        1,       

In [201]:
## mask_i : where to discard leftovers 
filtered_mask = ensemble_filtered == 32000
# Determine which columns have all rows as -1
columns_with_all_negative_one = torch.all(filtered_mask, dim=0)

# Get the first index where all rows have -1
nonzero_indices = torch.nonzero(columns_with_all_negative_one, as_tuple=True)

# If there are any such columns, find the first one
if len(nonzero_indices[0]) > 0:
    mask_k_i = nonzero_indices[0][0].item()
    k_final = min(mask_k_i, ensemble_indices_k_size)
else:
    mask_k_i = -1  # If no such index is found
    k_final = ensemble_indices_k_size

print(filtered_mask.shape)
print(filtered_mask)
print(columns_with_all_negative_one)
print(nonzero_indices)
print(nonzero_indices[0].shape)


torch.Size([6, 15])
tensor([[False,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True],
        [False,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True],
        [False,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True]])
tensor([False,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True])
(tensor([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14]),)
torch.Size([14])


In [202]:
mask_k_i, ensemble_indices_k_size

(1, 15)

In [203]:
k_final = min(mask_k_i, ensemble_indices_k_size)
ensemble_filtered = ensemble_filtered[:, :k_final]
ensemble_cnt_filtered = ensemble_cnt_filtered[:, :k_final]
ensemble_filtered = ensemble_filtered.view(_N_H, TDST_BQ, -1)

ensemble_filtered, ensemble_cnt_filtered

(tensor([[[    7],
          [32000],
          [   10]],
 
         [[    9],
          [32000],
          [32000]]]),
 tensor([[    4],
         [32000],
         [    4],
         [    4],
         [32000],
         [32000]]))

In [204]:
print(ensemble_attn_mask_per_layer)

tensor([[ 4, 11,  7,  9,  7,  7,  9,  1,  3,  7,  5, 10, 10,  3,  2],
        [ 9,  5,  3,  9,  8,  2,  1, 10,  3,  5,  4,  2,  8,  8,  9],
        [10,  3,  5,  4, 10, 10,  1, 10,  2,  7,  2,  7, 11,  7,  6],
        [ 9,  6,  7,  8,  9,  2,  8,  9,  4,  6, 11,  5, 10,  9,  6],
        [ 7, 11, 11,  7,  1,  8,  9,  5,  6,  7, 10,  6,  3,  1,  5],
        [ 1,  4,  9,  8, 10,  8, 10,  4, 11,  7,  8,  3,  6,  4,  5]])


In [205]:
k_mask = ensemble_filtered < 32000
ks = k_mask.sum(dim=-1).view(_N_H, TDST_BQ)
sparsity_per_layer = torch.sum(ensemble_filtered<32000).item()
sparsity_ratio = (sparsity_per_layer/origin_sparsity)


ensemble_filtered.shape, ensemble_filtered, k_mask, ks, origin_sparsity, sparsity_per_layer, sparsity_ratio, 

(torch.Size([2, 3, 1]),
 tensor([[[    7],
          [32000],
          [   10]],
 
         [[    9],
          [32000],
          [32000]]]),
 tensor([[[ True],
          [False],
          [ True]],
 
         [[ True],
          [False],
          [False]]]),
 tensor([[1, 0, 1],
         [1, 0, 0]]),
 18,
 3,
 0.16666666666666666)

In [216]:
# ensemble_attn_mask_per_layer [40, 128, 256, 5] to [N*H * TDST//BLOCK_SIZE_Q, mask_k//BLOCK_SIZE_K * ensemble_model_n]
import random
import numpy as np
import torch
import os
import math

# from ...hip.utils import seed
def seed(seed=50):
    torch.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed) # if use multi-GPU
    os.environ["PYTHONHASHSEED"] = str(seed)

ensemble_method_final_inter_thresh = 2
ensemble_method_final_bdd_mask_k = 0
ensemble_model_n = 5
seed()

ensemble_attn_mask_per_layer = torch.randint(1, 12, (2,3,3,5))
print(ensemble_attn_mask_per_layer.shape)

print(ensemble_attn_mask_per_layer)
_N_H, TDST_BQ, MASK_K_BK, MODEL_N = ensemble_attn_mask_per_layer.shape

origin_sparsity = (torch.sum(ensemble_attn_mask_per_layer < 32000)//ensemble_model_n).item()

if ensemble_method_final_bdd_mask_k == 1:
    pass
    # ensemble_indices_k_size = mask_k//block_size_k
else:
    # TODO set to max possible memory : change it more efficiently
    ensemble_indices_k_size = MASK_K_BK * MODEL_N 

ensemble_attn_mask_per_layer = ensemble_attn_mask_per_layer.view(_N_H * TDST_BQ, MASK_K_BK * MODEL_N)
print(ensemble_attn_mask_per_layer.shape)
print(ensemble_attn_mask_per_layer)
# TODO: Is it better to start plain and concatenate?
# ensembled_indices = torch.full((_N_H*TDST_BQ, ensemble_indices_k_size), 32000) # change to (N_H, TDST_BQ, ensemble_indices_k_size)

# k_size_max = 0

# NOTE per_query_token_cnt_diclist is just for analysis
if os.environ.get('ENSEMBLE_AGREE_DICLIST', '0') == '1':
    per_query_token_cnt_diclist = []

# ensemble_attn_mask_per_layer [40, 128, 256, 5] to [N*H * TDST//BLOCK_SIZE_Q, mask_k//BLOCK_SIZE_K * ensemble_model_n]
# ensemble_attn_mask_per_layer = ensemble_attn_mask_per_layer.view(_N_H * TDST_BQ, MASK_K_BK * MODEL_N)
# ensemble_attn_mask_per_layer : [N*H * TDST//BLOCK_SIZE_Q, mask_k//BLOCK_SIZE_K * ensemble_model_n]
unique_x, indices, unique_cnt = torch.unique(ensemble_attn_mask_per_layer, return_inverse=True, sorted=False, return_counts=True)
# indices -= indices.min(dim=1, keepdims=True)[0]

# cnt_x = (unique_x[None, None, :] == ensemble_attn_mask_per_layer[:, :, None]).long().sum(1)
N, K = ensemble_attn_mask_per_layer.shape

cnt_xs = []
chunk_n = 64
for icn in range(int(math.ceil(N / chunk_n))):
    cnt_xs.append(
        (unique_x[None, None, :] == ensemble_attn_mask_per_layer[icn*chunk_n:icn*chunk_n+chunk_n, :, None]).long().sum(1)
    )
cnt_x = torch.cat(cnt_xs, dim=0)


result = torch.full_like(ensemble_attn_mask_per_layer, 32000)
result = result.scatter_(1, indices.clamp(0, K-1), ensemble_attn_mask_per_layer)


# t = result[:, :, None] == unique_x[None, None, :]
# t = t * torch.arange(len(unique_x), device=t.device)[None, None, :]
# t = t.sum(-1)
N, K = result.shape
ts = []
chunk_n = 64
for icn in range(int(math.ceil(N / chunk_n))):
    t = result[icn*chunk_n:icn*chunk_n+chunk_n, :, None] == unique_x[None, None, :]
    t = t * torch.arange(len(unique_x), device=t.device)[None, None, :]
    t = t.sum(-1)
    ts.append(t)
t = torch.cat(ts, dim=0)

# torch.Size([4096, 1280]) torch.Size([64, 1280]) torch.Size([4096, 2094])
# print(result.shape, t.shape, cnt_x.shape)
result_cnt = torch.where(result < 32000, cnt_x.gather(-1, t), -9999999)

'''
ensemble_attn_mask_per_layer
tensor([[1, 1, 3, 3, 3, 5],\
        [3, 3, 4, 4, 4, 4]])
ensemble_cnt_sorted
tensor([[       6,        4,        3,        3,        2,        1,        1,
    -9999999, -9999999, -9999999, -9999999, -9999999, -9999999, -9999999,
    -9999999, -9999999, -9999999, -9999999, -9999999, -9999999],\
    [       6,        4,        3,        3,        2,        2, -9999999,
    -9999999, -9999999, -9999999, -9999999, -9999999, -9999999, -9999999,
    -9999999, -9999999, -9999999, -9999999, -9999999, -9999999]])
ensemble_sorted
    tensor([[   10,     6,     3,     8,     1,     5,    11, 32000, 32000, 32000,
    32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000],\
    [    5,     4,     7,     9,     6,     3, 32000, 32000, 32000, 32000,
    32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000]]))
'''
# ensemble_sorted : [N*H * TDST//BLOCK_SIZE_Q, mask_k//BLOCK_SIZE_K * ensemble_model_n]
# ensemble_cnt_sorted : [N*H * TDST//BLOCK_SIZE_Q, mask_k//BLOCK_SIZE_K * ensemble_model_n]
ensemble_cnt_sorted, indices = torch.sort(result_cnt, dim=-1, descending=True)
ensemble_sorted = result.gather(-1, indices)
mask = ensemble_cnt_sorted >= ensemble_method_final_inter_thresh

ensemble_filtered = torch.where(mask, ensemble_sorted, torch.tensor(32000, device=mask.device))
ensemble_cnt_filtered = torch.where(mask, ensemble_cnt_sorted, torch.tensor(32000, device=mask.device))

## mask_i : where to discard leftovers 
filtered_mask = ensemble_filtered == 32000
# Determine which columns have all rows as -1
columns_with_all_negative_one = torch.all(filtered_mask, dim=0)

# Get the first index where all rows have -1
nonzero_indices = torch.nonzero(columns_with_all_negative_one, as_tuple=True)

# If there are any such columns, find the first one
if len(nonzero_indices[0]) > 0:
    mask_k_i = nonzero_indices[0][0].item()
    k_final = min(mask_k_i, ensemble_indices_k_size)
else:
    mask_k_i = -1  # If no such index is found
    k_final = ensemble_indices_k_size

ensemble_filtered = ensemble_filtered[:, :k_final] # TODO is this meaningful??
ensemble_cnt_filtered = ensemble_cnt_filtered[:, :k_final]
ensemble_filtered = ensemble_filtered.view(_N_H, TDST_BQ, -1)

k_mask = ensemble_filtered < 32000
ks = k_mask.sum(dim=-1).view(_N_H, TDST_BQ)
sparsity_per_layer = torch.sum(ensemble_filtered<32000).item()
sparsity_ratio = (sparsity_per_layer/origin_sparsity)

print(ensemble_filtered)
print(ensemble_cnt_filtered)
print(ks)
print(sparsity_ratio)

torch.Size([2, 3, 3, 5])
tensor([[[[ 4, 11,  7,  9,  7],
          [ 7,  9,  1,  3,  7],
          [ 5, 10, 10,  3,  2]],

         [[ 9,  5,  3,  9,  8],
          [ 2,  1, 10,  3,  5],
          [ 4,  2,  8,  8,  9]],

         [[10,  3,  5,  4, 10],
          [10,  1, 10,  2,  7],
          [ 2,  7, 11,  7,  6]]],


        [[[ 9,  6,  7,  8,  9],
          [ 2,  8,  9,  4,  6],
          [11,  5, 10,  9,  6]],

         [[ 7, 11, 11,  7,  1],
          [ 8,  9,  5,  6,  7],
          [10,  6,  3,  1,  5]],

         [[ 1,  4,  9,  8, 10],
          [ 8, 10,  4, 11,  7],
          [ 8,  3,  6,  4,  5]]]])
torch.Size([6, 15])
tensor([[ 4, 11,  7,  9,  7,  7,  9,  1,  3,  7,  5, 10, 10,  3,  2],
        [ 9,  5,  3,  9,  8,  2,  1, 10,  3,  5,  4,  2,  8,  8,  9],
        [10,  3,  5,  4, 10, 10,  1, 10,  2,  7,  2,  7, 11,  7,  6],
        [ 9,  6,  7,  8,  9,  2,  8,  9,  4,  6, 11,  5, 10,  9,  6],
        [ 7, 11, 11,  7,  1,  8,  9,  5,  6,  7, 10,  6,  3,  1,  5],
        [ 1,  